In [ ]:
import docker
import json
import subprocess
import os

In [ ]:
def reassign_kafka_partitions(topics, broker_list, bootstrap_server):
    #Generate JSON for topics
    topics_json = {"topics": [{"topic": name} for name in topics], "version": 1}
    topics_json_path = 'change_topics.json'
    with open(topics_json_path, 'w') as file:
        json.dump(topics_json, file, indent=2)
    
    generate_command = [
        '/bin/kafka-reassign-partitions',
        '--bootstrap-server', bootstrap_server,
        '--topics-to-move-json-file', topics_json_path,
        '--broker-list', broker_list,
        '--generate'
    ]
    
    try:
        generate_output = subprocess.run(generate_command, check=True, text=True, capture_output=True)
        proposed_config = generate_output.stdout.split("Proposed partition reassignment configuration:")[-1]
        proposed_config = proposed_config.split("Current partition replica assignment")[0].strip()
        
        reassignment_json_path = 'reassignment.json'
        with open(reassignment_json_path, 'w') as file:
            file.write(proposed_config)
        
        execute_command = [
            '/bin/kafka-reassign-partitions',
            '--bootstrap-server', bootstrap_server,
            '--reassignment-json-file', reassignment_json_path,
            '--execute'
        ]
        
        execute_output = subprocess.run(execute_command, check=True, text=True, capture_output=True)
        if "successfully" in execute_output.stdout:
            print("Partition reassignment succeeded.")
        else:
            print("Partition reassignment failed.")
            print(execute_output.stdout)
        
    except subprocess.CalledProcessError as e:
        print(f"Command '{e.cmd}' returned non-zero exit status {e.returncode}.")
        print(e.output)
    
    finally:
        #Cleanup
        if os.path.exists(topics_json_path):
            os.remove(topics_json_path)
        if os.path.exists(reassignment_json_path):
            os.remove(reassignment_json_path)

In [ ]:
#EXAMPLE USAGE
reassign_kafka_partitions(["topic1", "topic2"], "1,2,3", "localhost:9092")